In [1]:
import requests
import json
from bs4 import BeautifulSoup

import ast
import pandas as pd
import csv
from csv import DictWriter, DictReader

import os
from PIL import Image

In [15]:
main()

Page 7
Page 8
Page 9
Page 10
Page 11
Page 12
Page 13
Page 14
Page 15


In [ ]:
# ambil data tambahan dari page new release per genre
def get_new_release(page, dir_path, header, cookies):
    request = requests.get(page, headers=header, cookies=cookies)
    
    soup = BeautifulSoup(request.text, 'html.parser')
    row = soup.find_all('div', class_='coverWrapper')
    
    for r in row:
        href = r.find('a')['href']
        
        single_page = 'http://www.goodreads.com'+href
        single_req = requests.get(single_page, headers=header, cookies=cookies)
        
        single_soup = BeautifulSoup(single_req.text, 'html.parser')
        get_data_each_book(single_soup, dir_path)

In [ ]:
# cari buku dari isbn di folder lama (untuk perbaikan data training)
def search_book(file_dir, save_dir, header, cookies):
    file_isbn = os.listdir(file_dir)
    
    for file in file_isbn:
        isbn = file.strip('.jpg .png')
        
        if file not in os.listdir(save_dir):
#             print(isbn)
            len_string = str(len(isbn))
            url = 'https://www.goodreads.com/search?q='+isbn+'&ref=nav_sb_noss_l_'+len_string
            request = requests.get(url, headers=header, cookies=cookies)

            soup = BeautifulSoup(request.text,'html.parser')
            tag = soup.find('h3', class_='searchSubNavContainer')

            if tag is None:
                get_data_each_book(soup, save_dir)
            elif tag.text=='No results.':
                pass
        else:
#             os.remove(file_dir+'/'+file)
            pass

In [2]:
def get_best_book(page, csv_name, header, cookies): 
    request = requests.get(page, headers=header, cookies=cookies) 
    
    soup = BeautifulSoup(request.text, 'html.parser')
    row = soup.find_all('td', attrs={'width':'100%'})
    
    for r in row:
        href = r.find('a', class_='bookTitle')['href']
        
        single_page = 'https://www.goodreads.com'+href
        request = requests.get(single_page, headers=header, cookies=cookies)
        single_soup = BeautifulSoup(request.text, 'html.parser')
        
        get_data_each_book(single_soup, csv_name)

In [3]:
# get book data from each page
def get_data_each_book(soup, csv_name):#, save_dir): #dir_path):
    title = soup.find('h1', class_='Text Text__title1').text
    cover = soup.find('img', class_='BookCover__image')['src']
    isbn = get_isbn(soup)
    
    # untuk yg best_book
    link = soup.find('img', class_='BookCover__image')['data-csa-c-content-id']
    genre = get_genre(soup, link)
    
    to_csv_best_book(title, isbn, genre, cover, csv_name)
    
#     # untuk yg search_book
#     print(isbn+'  __  '+cover)
#     if cover!='':
#         save_image(isbn, cover, save_dir)
#     else:
#         pass

In [4]:
# get book's isbn
def get_isbn(soup):
    script = soup.find('script', attrs={'type':'application/ld+json'})
    json_object = json.loads(script.contents[0])
    
    if 'isbn' not in json_object:
        isbn = (json_object['image'].split('/')[-1]).strip('.jpg')
    else:
        isbn = json_object['isbn']
    return isbn

In [5]:
# get book's genre 
def get_genre(soup, link):
    script = soup.find('script', attrs={'id':'__NEXT_DATA__'})
    json_object = json.loads(script.contents[0])
    json_genre = json_object['props']['pageProps']['apolloState']['Book:'+link]['bookGenres']
    pd = []
    
    for genre_list in json_genre:
        genre = genre_list['genre']['name']
        
        if (genre=='Fantasy' or genre=='Science Fiction' or genre=='Epic Fantasy' or genre=='Urban Fantasy' or genre=='Dark Fantasy' or genre=='Fairy Tales'
            or genre=='General' or genre=='Literary Fiction' or genre=='Literature' or genre=='General Fiction' or genre=='Contemporary' 
            or genre=='Realistic Fiction' or genre=='Coming of Age' or genre=='Horror' or genre=='Paranormal' or genre=='Gothic Horror' 
            or genre=='Ghost Stories' or genre=='Ghost' or genre=='Mystery' or genre=='Thriller' or genre=='Mystery Thriller' 
            or genre=='Psychological Thriller' or genre=='Historical Mystery' or genre=='Detective' or genre=='Murder Mystery' or genre=='Romance' 
            or genre=='Historical Romance' or genre=='Contemporary Romance'):
            pd.append(genre)
    return pd

In [6]:
# save title, isbn, and cover url to file csv
def to_csv_best_book(title, isbn, genre, cover, csv_name):
    fields = ['title', 'isbn', 'genre', 'url_cover']
    bookcsv = {'title':title, 'isbn':{isbn}, 'genre': genre, 'url_cover':cover}

    if not os.path.exists(csv_name):
        with open(csv_name, 'a+', encoding='utf-8', newline='') as csvfile:
            writer = DictWriter(csvfile, fieldnames=fields)
            writer.writeheader()
            writer.writerow(bookcsv)
    else:
        with open(csv_name, 'a+', encoding='utf-8', newline='') as csvfile:
            writer = DictWriter(csvfile, fieldnames=fields)
            writer.writerow(bookcsv)

In [7]:
def to_csv_genre(csv_source, cvs_genre): 
    with open(csv_source) as csvfile:
        reader = DictReader(csvfile)
        for row in reader:
            f = h = g = m = r = 0
            
            title = row['title']
            isbn = row['isbn']
            genre_list = row['genre']
            url = row['url_cover']
            
            if genre_list!='[]' or url!='[]':
                genres = ast.literal_eval(genre_list)

                for genre in genres:
                    if(genre=='Fantasy' or genre=='Science Fiction' or genre=='Epic Fantasy' or genre=='Urban Fantasy' or genre=='Dark Fantasy' 
                       or genre=='Fairy Tales'):
                        f=1
                    elif(genre=='General' or genre == 'Literature'  or genre=='Literary Fiction' or genre=='General Fiction' or genre=='Contemporary' 
                          or genre=='Realistic Fiction' or genre=='Coming of Age'):
                        g=1
                    elif(genre=='Horror' or genre=='Paranormal' or genre=='Gothic Horror' or genre=='Ghost Stories' or genre=='Ghost'):
                        h=1
                    elif(genre=='Mystery' or genre=='Thriller' or genre=='Mystery Thriller' or genre=='Psychological Thriller' or genre=='Historical Mystery' 
                          or genre=='Detective' or genre=='Murder Mystery'):
                        m=1
                    elif(genre=='Romance' or genre=='Historical Romance' or genre=='Contemporary Romance'):
                        r=1

                fields = ['title', 'isbn', 'fantasy', 'general', 'horror', 'mystery', 'romance', 'url_cover']
                bookcsv = {'title':title, 'isbn':isbn, 'fantasy':f, 'general':g, 'horror':h, 'mystery':m, 'romance':r, 'url_cover':url}

                if not os.path.exists(csv_genre):
                    with open(csv_genre, 'a+', encoding='utf-8', newline='') as csvfile:
                        writer = DictWriter(csvfile, fieldnames=fields)
                        writer.writeheader()
                        writer.writerow(bookcsv)
                else:
                    with open(csv_genre, 'a', encoding='utf-8', newline='') as csvfile:
                        writer = DictWriter(csvfile, fieldnames=fields)
                        writer.writerow(bookcsv)
            
            else:
                print('Skip '+title+' genre atau cover null.')

In [8]:
# load url cover to save cover image to device with isbn as the name
def load_from_csv(csv_name, dir_path):
    with open(csv_name) as csvfile:
        reader = DictReader(csvfile)
        for row in reader:
            isbn = row['isbn']
            cover = row['url_cover']
            
            save_image(isbn, cover, dir_path)

In [9]:
# save book cover image 
def save_image(isbn, url, dir_path):
    filename = isbn+'.jpg' 
    folder_path = dir_path + filename
    
    if filename not in os.listdir(dir_path):
        req = requests.get(url, stream=True)
        if req.status_code==200:
            image = Image.open(req.raw)
            image.convert('RGB').save(folder_path, quality=100)
        else:
            print('-----------------------------------------Hapus isbn '+isbn+' di bestbook.csv!--------------------------------------------')

In [ ]:
# resize book cover image then save the resized file, filename='isbn.jpg'
def resize_image(filename, width, dir_path):
    resize_path = dir_path+filename
    
    img = Image.open(filename)
    img = img.resize((width, width), Image.ANTIALIAS)
    img.save(resize_path, quality=100)

In [ ]:
# save title, isbn, genres to csv for genre label
def csv_genre(csv_genre):
    fields = ['title', 'isbn', 'fantasy', 'horror', 'literary_fiction', 'mystery', 'romance']
    bookcsv = {'title':title, 'isbn':isbn, 'fantasy':f, 'horror':h, 'literary_fiction':lf, 'mystery':m, 'romance':r}

    if not os.path.exists(csv_genre):
        with open(csv_genre, 'a+', newline='') as csvfile:
            writer = DictWriter(csvfile, fieldnames=fields)
            writer.writeheader()
            writer.writerow(bookcsv)
    else:
        with open(csv_genre, 'a', newline='') as csvfile:
            writer = DictWriter(csvfile, fieldnames=fields)
            writer.writerow(bookcsv)

In [14]:
def main():#genre):
    cookies = {
        'p': '7MwzXHKHiVxkh8eUg0yexRqR7j7jgidcU_xc65ih_FatI2-l',
        'ccsid': '612-0793395-0086739',
        'locale': 'en',
        '__qca': 'P0-1068727077-1635475725541',
        'logged_out_browsing_page_count': '2',
        'u': '3tKr8P1RiuWXO-hWNd8ai0HlqjjSAcsntLil-R8sqMuuULuD',
        'likely_has_account': 'true',
        'csm-sid': '351-9527432-3091124',
        '_session_id2': 'c6a4485b29a791378d4f8dd83e0d5f5b',
        'srb_1': '1_wl',
    }

    headers = {
        'Connection': 'keep-alive',
        'Cache-Control': 'max-age=0',
        'sec-ch-ua': '"Google Chrome";v="95", "Chromium";v="95", ";Not A Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'DNT': '1',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-User': '?1',
        'Sec-Fetch-Dest': 'document',
        'Referer': 'https://www.goodreads.com/book/show/51457170-blood-heir',
        'Accept-Language': 'en-US,en;q=0.9,id;q=0.8',
        'If-None-Match': 'W/"47657-on/6fBsXMVqOkNwnZqCifG3TsDs"',
    }
    
#     file_dir = 'train/general'
#     save_dir = 'test/'
#     search_book(file_dir, save_dir, headers, cookies)
    
#     new_release = 'https://www.goodreads.com/genres/new_releases/'+genre
#     nr_dir = 'new_release/'
#     get_new_release(new_release, nr_dir, headers, cookies)
    
    csv_best_book = 'best_book_2018.csv'
    csv_test = 'test.csv'
    test_dir = 'test/'
    
    for i in range(7,16):
        page = 'https://www.goodreads.com/list/show/119307.Best_Books_of_2018?page='+str(i)
        get_best_book(page, csv_best_book, headers, cookies)
        print('Page '+str(i))
    
#     to_csv_genre(csv_best_book, csv_test)
    
#     load_from_csv(csv_test, test_dir)